## Some Instruction before you run this notebook.
1. Change the line 29 of dataloader_clustering module from wsi_id = wsi_folder to wsi_id = wsi_file[:12]
2. Edit the second cell configurations according to your paths
3. Use patch level features for this. (Fivecrops or Patch Level Averaged)  

In [ ]:
import torch
import torchvision
from torch.utils.data import DataLoader
import os
from os.path import join as j_
from PIL import Image
import pandas as pd
import numpy as np
import time
import random
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import Lambda
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
# loading all packages here to start
from dataloader_clustering import WSIDataset
from eval_patch_features.logistic import eval_linear
from eval_patch_features.ann import eval_ANN
from eval_patch_features.knn import eval_knn
from eval_patch_features.protonet import eval_protonet
from eval_patch_features.metrics import get_eval_metrics, print_metrics
from utility import calculate_metric_averages, average_confusion_matrices, write_data_in_excel, build_probs_df
import warnings
warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


### Configurations

In [ ]:
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
# configs
VECTOR_DIM = 512  # size of input feature vector
HIDDEN_DIM = 768   # size of ANN hidden layer
BATCH_SIZE = 8
FM_MODEL = "baseline"
RUNS_RESULT = "average"
ANN_RUNS = 20
CLUSTERING_METHOD = 'kmeans'
NUM_CLUSTERS = 2
NUM_PATCHES_PER_CLUSTER = 0
K_FOLDS_PATH = r"E:\KSA Project\dataset\splits\kfolds.csv"
DATA_PATH = f"E:\KSA Project\dataset\{FM_MODEL}_fivecrop_features"
MODEL_SAVE_PATH = f"E:\KSA Project\KSAproject_pipeline1\WSI_Classification\Clustering\TCGA-CV\{FM_MODEL}_2Cluster_Classifiers"
os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
OUTPUT_SAVE_PATH = r"E:\KSA Project\KSAproject_pipeline1\WSI_Classification\Clustering\TCGA-CV"
os.makedirs(OUTPUT_SAVE_PATH, exist_ok=True)
# create a excel sheet in the output folder to save the results
EVAL_METRICS_EXCEL = os.path.join(OUTPUT_SAVE_PATH, "TCGA-CV_2cluster_eval_metrics.xlsx")
PROBS_ALL_EXCEL = os.path.join(OUTPUT_SAVE_PATH, "TCGA-CV_2cluster_probs_all.xlsx")

## Trainer Function

In [ ]:
def train_and_evaluate(fold,train_loader,val_loader, test_loader, model_type='linear'):
    all_train_feats, all_train_labels,all_val_feats,all_val_labels, all_test_feats, all_test_labels = [], [], [], [], [], []
    all_test_ids = []
    # Prepare training and testing data
    for features, label, _ in train_loader:
        all_train_feats.append(features)
        all_train_labels.append(label)
    for features, label, _ in val_loader:
        all_val_feats.append(features)
        all_val_labels.append(label)
    for features, label, wsi_id in test_loader:
        all_test_feats.append(features)
        all_test_labels.append(label)
        # Store as single WSI IDs from the batch 
        if isinstance(wsi_id, (list, tuple)):
            all_test_ids.extend(wsi_id)
        else:
            all_test_ids.append(wsi_id)

    # Convert lists to tensors
    global train_feats, train_labels, val_feats, val_labels, test_feats, test_labels
    train_feats = torch.cat(all_train_feats)
    train_labels = torch.cat([labels.clone().detach() for labels in all_train_labels])
    val_feats = torch.cat(all_val_feats)
    val_labels = torch.cat([labels.clone().detach() for labels in all_val_labels])
    test_feats = torch.cat(all_test_feats)
    test_labels = torch.cat([labels.clone().detach() for labels in all_test_labels])
    
    # Select the model based on the input argument
    if model_type == 'lin':
        eval_metrics, eval_dump = eval_linear(
            fold=fold,
            train_feats=train_feats,
            train_labels=train_labels,
            valid_feats=val_feats,  # Optionally, use a separate validation set
            valid_labels=val_labels,
            test_feats=test_feats,
            test_labels=test_labels,
            max_iter=350,
            save_path = MODEL_SAVE_PATH,
            verbose=False,
        )
    elif model_type == 'ann':
        eval_metrics, eval_dump = eval_ANN(
            fold=fold,
            train_feats=train_feats,
            train_labels=train_labels,
            valid_feats=val_feats,
            valid_labels=val_labels,
            test_feats=test_feats,
            test_labels=test_labels,
            input_dim=VECTOR_DIM * NUM_CLUSTERS,
            hidden_dim = HIDDEN_DIM,
            max_iter=350,
            model_save_path = MODEL_SAVE_PATH,
            num_runs=ANN_RUNS,  # Run the function 5 times
            runs_results=RUNS_RESULT,  # Choose "average" or "best"
            metric_weights={"bacc": 0.5, "auroc": 0.5},  # Prioritize balanced accuracy
            verbose=False,
        )
    elif model_type == 'knn':
        eval_metrics, eval_dump = eval_knn(
            fold=fold,
            train_feats=train_feats,
            train_labels=train_labels,
            val_feats=val_feats,
            val_labels=val_labels,
            test_feats=test_feats,
            test_labels=test_labels,
            n_neighbors=5,
            normalize_feats=True,
            model_save_path = MODEL_SAVE_PATH,
            verbose=False

        )
    elif model_type == 'proto':
        eval_metrics, eval_dump = eval_protonet(
            fold=fold,
            train_feats=train_feats,
            train_labels=train_labels,
            val_feats=val_feats,
            val_labels=val_labels,
            test_feats=test_feats,
            test_labels=test_labels,
            normalize_feats=True,
            model_save_path = MODEL_SAVE_PATH,
        )
        
    else:
        raise ValueError(f"Unsupported model type: {model_type}")
    return eval_metrics, eval_dump, all_test_ids

### K-Folds

In [ ]:
from typing import List
def run_k_fold_cross_validation(save_dir: str, folds: List[List[str]], model_type: str = 'lin'):
    results_per_fold = []
    num_folds = len(folds)

    for i in range(num_folds):
        # Define test and validation folds
        test_ids = folds[i]
        val_ids = folds[(i + 1) % num_folds]  # The next fold in sequence is used as validation

        # Use remaining folds as training
        train_ids = []
        for j in range(num_folds):
            if j != i and j != (i + 1) % num_folds:
                train_ids.extend(folds[j])
        print(f"Running Fold {i + 1} with model {model_type}...")
        # Create datasets and loaders
        train_dataset = WSIDataset(save_dir, train_ids)
        train_dataset.apply_clustering(clustering_algorithm=CLUSTERING_METHOD, num_clusters=NUM_CLUSTERS, num_selected_patches=NUM_PATCHES_PER_CLUSTER)
        val_dataset = WSIDataset(save_dir, val_ids)
        val_dataset.apply_clustering(clustering_algorithm=CLUSTERING_METHOD, num_clusters=NUM_CLUSTERS, num_selected_patches=NUM_PATCHES_PER_CLUSTER)
        test_dataset = WSIDataset(save_dir, test_ids)
        test_dataset.apply_clustering(clustering_algorithm=CLUSTERING_METHOD, num_clusters=NUM_CLUSTERS, num_selected_patches=NUM_PATCHES_PER_CLUSTER)
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
        eval_metrics, eval_dump, all_test_ids = train_and_evaluate(i,train_loader,val_loader, test_loader, model_type=model_type)
        print_metrics(eval_metrics)
        result = {
            **eval_metrics,
            **eval_dump,
            "wsi_ids":all_test_ids ,  # You already have this in train_and_evaluate
            "fold": i + 1
        }
        results_per_fold.append(result)
    return results_per_fold

### Main Runner Function

In [ ]:
# Example usage:
folds_df = pd.read_csv(K_FOLDS_PATH)
# Define your folds
fold1_ids = folds_df['Fold1'].dropna().apply(lambda x: x[:12]).tolist()
fold2_ids = folds_df['Fold2'].dropna().apply(lambda x: x[:12]).tolist()
fold3_ids = folds_df['Fold3'].dropna().apply(lambda x: x[:12]).tolist()
fold4_ids = folds_df['Fold4'].dropna().apply(lambda x: x[:12]).tolist()
folds = [fold1_ids, fold2_ids, fold3_ids, fold4_ids]

# Run k-fold cross-validation with different models
model_types = ['lin','ann','knn','proto']
metric_indices = {
    'acc': 0,          
    'bacc': 1,      
    'macro_f1': 2,     
    'weighted_f1': 3, 
    'auroc': 4      
}
eval_metrics__for_excel = []
probs_all_for_excel = None
for model in model_types:
    predictions_list = []
    print(f"\n\n ********* Training with model: {model}********* \n\n")
    k_folds_results = run_k_fold_cross_validation(DATA_PATH, folds, model_type=model)
    model_df = build_probs_df(k_folds_results,model_name=model)
    # === Merge predictions across models ===
    if probs_all_for_excel is None:
        probs_all_for_excel = model_df
    else:
        probs_all_for_excel=pd.merge(probs_all_for_excel,model_df,on=["Fold", "WSI_ID", "Target"],how="outer")

    # === Average metrics (only pass metric parts of result dicts)
    average_results = calculate_metric_averages(
        [{k: v for k, v in result.items() if k in [f"{model}_{m}" for m in metric_indices.keys()]}
        for result in k_folds_results],
        metric_indices,
        model_prefix=model
    )
    # === Confusion matrices
    confusion_matrices = [np.array(result[f"{model}_conf_matrix"]) for result in k_folds_results if f"{model}_conf_matrix" in result]
    
    avg_conf_matrix = average_confusion_matrices(confusion_matrices)
    print("\n\n Average results for all folds:")
    for metric, value in average_results.items():
        print(f"{metric}: {value:.4f}")
     # Append per metric rows for each fold + average
    for metric in metric_indices.keys():
        row = [f"{model}_{metric}"]
        for result in k_folds_results:
            row.append(result.get(f"{model}_{metric}", 'N/A'))
        row.append(average_results.get(f"{model}_{metric}", 'N/A'))
        eval_metrics__for_excel.append(row)

    # Append confusion matrix as string (per fold)
    row = [f"{model}_conf_matrix"]
    for result in k_folds_results:
        row.append(str(result.get(f"{model}_conf_matrix", "N/A")))
    row.append(str(avg_conf_matrix))
    eval_metrics__for_excel.append(row)
    
eval_metrics_df = pd.DataFrame(eval_metrics__for_excel, 
                        columns=["Metric","Fold1","Fold2","Fold3","Fold4","AvgFolds"])
write_data_in_excel(EVAL_METRICS_EXCEL, eval_metrics_df, FM_MODEL)
write_data_in_excel(PROBS_ALL_EXCEL, probs_all_for_excel, FM_MODEL)



 ********* Training with model: lin********* 


Running Fold 1 with model lin...
lin_acc: 0.7700
lin_bacc: 0.7549
lin_macro_f1: 0.6703
lin_weighted_f1: 0.7972
lin_auroc: 0.8204
lin_conf_matrix: [[66 19]
 [ 4 11]]
Running Fold 2 with model lin...
lin_acc: 0.8617
lin_bacc: 0.5000
lin_macro_f1: 0.4629
lin_weighted_f1: 0.7977
lin_auroc: 0.5347
lin_conf_matrix: [[81  0]
 [13  0]]
Running Fold 3 with model lin...
lin_acc: 0.7748
lin_bacc: 0.5896
lin_macro_f1: 0.5946
lin_weighted_f1: 0.7675
lin_auroc: 0.6538
lin_conf_matrix: [[80 11]
 [14  6]]
Running Fold 4 with model lin...
lin_acc: 0.9000
lin_bacc: 0.6808
lin_macro_f1: 0.7222
lin_weighted_f1: 0.8867
lin_auroc: 0.8214
lin_conf_matrix: [[85  2]
 [ 8  5]]


 Average results for all folds:
lin_acc: 0.8266
lin_bacc: 0.6313
lin_macro_f1: 0.6125
lin_weighted_f1: 0.8123
lin_auroc: 0.7076


 ********* Training with model: ann********* 


Running Fold 1 with model ann...
ann_acc: 0.4400
ann_bacc: 0.6157
ann_macro_f1: 0.4212
ann_weighted_f1: 0.4942